In [ ]:
import polars as pl
from Levenshtein import ratio

In [ ]:
terms = (
    pl.scan_csv("data/terms.csv", infer_schema_length=10000)
)

In [ ]:
gateway = (
    pl.scan_csv('data/gateway_connections.csv', infer_schema_length=10000)
)